Zad.1
Porównaj zapisywanie i odczytywanie kolekcji (100, 10000, 100 000 elementów) za pomocą trzech technik: modułu pickle, parquet i xlsx.

In [90]:
from openpyxl import Workbook, load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import pandas as pd
from pickle import dump, load
from tempfile import mktemp
from os import unlink
import time
from fastparquet import ParquetFile


keys100 = list(range(0,100))
for i in range(len(keys100)):
    keys100[i] = f'element{i}'
    
keys10000 = list(range(0,10000))
for i in range(len(keys10000)):
    keys10000[i] = f'element{i}'
    
keys100000 = list(range(0,100000))
for i in range(len(keys100000)):
    keys100000[i] = f'element{i}'

dict100 = dict.fromkeys(keys100, list(range(1,3)))
dict10000 = dict.fromkeys(keys10000, list(range(1,3)))
dict100000 = dict.fromkeys(keys100000, list(range(1,3)))
def Save_Load_Pickle(collection, file):
    with open(file, 'wb') as f:
        start = time.time()
        dump(collection, f)
        end = time.time()

        print(f'Czas operacji zapisu dla kolekcji o długości {len(collection)} w module pickle: ok. {round(end - start, 10)} s')

        f.flush()

    with open(file, 'rb') as f:
        sumOf = 0
        start = time.time()
        temp_col = load(f)
        end = time.time()

        print(f'Czas operacji odczytu dla kolekcji o długości {len(collection)} w module pickle: ok. {round(end - start, 10)} s')

temp_file = mktemp()
Save_Load_Pickle(dict100, temp_file)
unlink(temp_file)

temp_file = mktemp()
Save_Load_Pickle(dict10000, temp_file)
unlink(temp_file)

temp_file = mktemp()
Save_Load_Pickle(dict100000, temp_file)
unlink(temp_file)

def Save_Load_Parquet(collection, file):
    df = pd.DataFrame.from_dict(collection)
    
    start = time.time()
    df.to_parquet(file, compression='GZIP')
    end = time.time()

    print(f'Czas operacji zapisu dla kolekcji o długości {len(collection)} w module parquet: ok. {round(end - start, 10)} s')

    start = time.time()
    pf = ParquetFile(file)
    end = time.time()

    print(f'Czas operacji odczytu dla kolekcji o długości {len(collection)} w module parquet: ok. {round(end - start, 10)} s')


temp_file = mktemp()
Save_Load_Parquet(dict100, temp_file)
unlink(temp_file)

temp_file = mktemp()
Save_Load_Parquet(dict10000, temp_file)
unlink(temp_file)

temp_file = mktemp()
Save_Load_Parquet(dict100000, temp_file)
unlink(temp_file)

def Save_Load_Xlsx(collection, file):
    try:
        df = pd.DataFrame.from_dict(collection)
        with open(file, 'wb') as f:
            wb = Workbook()
            ws = wb.active

            for r in dataframe_to_rows(df, index=True, header=True):
                ws.append(r)

            for cell in ws['A'] + ws[1]:
                cell.style = 'Pandas'

            start = time.time()
            wb.save(f.name)
            end = time.time()

            print(f'Czas operacji zapisu dla kolekcji o długości {len(collection)} w module xlsx: ok. {round(end - start, 10)} s')

            f.flush()

        with open(file, 'rb') as f:
            start = time.time()
            wb = load_workbook(f.name)
            end = time.time()

            print(f'Czas operacji odczytu dla kolekcji o długości {len(collection)} w module xlsx: ok. {round(end - start, 10)} s')
    except Exception as e:
        print(f'\nWystąpił błąd podczas zapisu pliku {len(collection)} elementowego: {e}!')
        print(f'Wiadomość {e} oznacza, że nie można zapisać pliku o tak dużej ilości kolumn')
        
temp_file = mktemp(suffix='.xlsx')   
Save_Load_Xlsx(dict100, temp_file)
unlink(temp_file)

temp_file = mktemp(suffix='.xlsx')   
Save_Load_Xlsx(dict10000, temp_file)
unlink(temp_file)

temp_file = mktemp(suffix='.xlsx')   
Save_Load_Xlsx(dict100000, temp_file)
unlink(temp_file)
    

print('\n Podsumowanie:\n')
print('Pickle [odczyt szybszy niż zapis]\nMożemy zauważyć, że najkrótsze czasy odczytu i zapisu słowników 100, 10000 i 100000 elementowych\nwystępują dla modułu pickle, jest to związane z faktem, że ów modół zapisuje mniej skomplikowane dane do pliku (słownik),\nlecz jest najszybszy. Oczywiście nie umożliwia nam on zapisywania struktur bardziej złożonych jak: dataframe lub plik \".xlsx\".\n\nParquet [odczyt znacząco szybszy niż zapis]\nKolejny moduł - parquet umożliwia nam zapis i odczyt struktury dataframe, zapis zajmuje zdecydowanie dłużej czasu od pickle i \nxlsx, lecz jest bardzo wygodny do działań na obiektach dataframe. Umożliwia nam również kompresje pliku.\n\nXlsx [zapis szybszy niż odczyt]\nOstatni moduł - xlsx pozwala na zapis i odczyt plików \".xslx\", jest bardzo szybki w swoich działaniach, lecz nie pozwala nam\nzapisywać ogromnych danych ze względu na ograniczenie kolumn w pliku \".xslx\" - Invalid column index 100001.\n\nKażdy z powyższych modułów jest użyteczny na inny sposób i służy do innych czynności.')

Czas operacji zapisu dla kolekcji o długości 100 w module pickle: ok. 0.0 s
Czas operacji odczytu dla kolekcji o długości 100 w module pickle: ok. 0.0 s
Czas operacji zapisu dla kolekcji o długości 10000 w module pickle: ok. 0.0010027885 s
Czas operacji odczytu dla kolekcji o długości 10000 w module pickle: ok. 0.0009965897 s
Czas operacji zapisu dla kolekcji o długości 100000 w module pickle: ok. 0.0141186714 s
Czas operacji odczytu dla kolekcji o długości 100000 w module pickle: ok. 0.0156621933 s
Czas operacji zapisu dla kolekcji o długości 100 w module parquet: ok. 0.0868506432 s
Czas operacji odczytu dla kolekcji o długości 100 w module parquet: ok. 0.0 s
Czas operacji zapisu dla kolekcji o długości 10000 w module parquet: ok. 8.3829042912 s
Czas operacji odczytu dla kolekcji o długości 10000 w module parquet: ok. 0.1428835392 s
Czas operacji zapisu dla kolekcji o długości 100000 w module parquet: ok. 79.3076577187 s
Czas operacji odczytu dla kolekcji o długości 100000 w module pa